In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import confusion_matrix
from keras.layers import Dense, Activation, Dropout
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import random

from imblearn.under_sampling import RandomUnderSampler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import GridSearchCV

from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.utils.class_weight import compute_class_weight
import joblib

In [2]:
df1 = pd.read_csv('/Users/ohidabinteamin/Documents/Stress Prediction Project Three Datasets/StudentLife/week 01/Social_Interaction/recreating_social_features_studentlife.csv')
df1 = df1.rename(columns={'Date': 'date'})
df1 = df1.drop('Unnamed: 0', axis=1)
df2 = pd.read_csv('/Users/ohidabinteamin/Documents/Stress Prediction Project Three Datasets/StudentLife/week 01/Stress/recreating_dailystress_features.csv')
df2 = df2.drop('Unnamed: 0', axis=1)

In [3]:
df = pd.merge(df1, df2, on=['uid', 'date'])
print(df.columns)

df = df.sort_values(by='date')

Index(['Unnamed: 0.1', 'uid', 'date', 'app usage in morning',
       'app usage in afternoon', 'app usage in evening', 'app usage in night',
       'number of call in morning', 'number of call in afternoon',
       'number of call in evening', 'number of call in night',
       'number of Bluetooth contacts morning',
       'number of Bluetooth contacts afternoon',
       'number of Bluetooth contacts evening',
       'number of Bluetooth contacts night', 'conversation in morning',
       'conversation in afternoon', 'conversation in evening',
       'conversation in night', 'stress_ratings'],
      dtype='object')


In [4]:
df = df.dropna()
print(len(df))

495


In [5]:
df.columns
del df['Unnamed: 0.1']

In [6]:
df['stress_ratings'].value_counts()

stress_ratings
medium stress    193
high stress      162
low stress       140
Name: count, dtype: int64

In [7]:
binary_lh_data = df[df['stress_ratings'].isin(['low stress', 'high stress'])]

In [8]:
X = binary_lh_data.drop(columns=['stress_ratings', 'uid', 'date'])
y = binary_lh_data['stress_ratings']
groups = binary_lh_data['uid']

stress_map = {'low stress': 0, 'high stress': 1}
y_encoded = y.map(stress_map).values 

logo = LeaveOneGroupOut()

In [9]:
unique, counts = np.unique(y_encoded, return_counts=True)
result = dict(zip(unique, counts))

print(result)

{0: 140, 1: 162}


In [10]:
logo = LeaveOneGroupOut()

best_thresholds = []
balanced_accs = []
auc_scores = []

In [11]:
seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)

In [12]:
num_splits = len(np.unique(groups))
print(num_splits)

with tqdm(total=num_splits, desc="LOSO CV Progress", unit="fold", bar_format="{l_bar}{bar} | {n_fmt}/{total_fmt} [{elapsed}<{remaining}] {percentage:3.0f}%") as pbar:
    for train_idx, test_idx in logo.split(X, y_encoded, groups=groups):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y_encoded[train_idx], y_encoded[test_idx]  

        print('X and y train shapes: ')
        print(X_train.shape)
        print(y_train.shape)

        train_class_counts = np.bincount(y_train)
        print(f"Train set - Class 0 (Low Stress): {train_class_counts[0]}, "
              f"Class 1 (High Stress): {train_class_counts[1] if len(train_class_counts) > 1 else 0}")

        test_class_counts = np.bincount(y_test)
        print(f"Test set - Class 0 (Low Stress): {test_class_counts[0]}, "
              f"Class 1 (High Stress): {test_class_counts[1] if len(test_class_counts) > 1 else 0}")

        print(f"Test set size: {len(y_test)}")
        print(f"Train set shape: {X_train.shape}")
        print(f"Test set shape: {X_test.shape}")

        scaler = StandardScaler()
        X_train_normalized = scaler.fit_transform(X_train)
        X_test_normalized = scaler.transform(X_test)

        print(X_train_normalized.shape)
        print(X_test_normalized.shape)

        # Define the FCN model
        model_fcn_binary_lh = tf.keras.models.Sequential([
            tf.keras.layers.Dense(57, activation='relu', input_dim=X_train_normalized.shape[1]),
            tf.keras.layers.Dropout(0.35),
            tf.keras.layers.Dense(35, activation='relu'),
            tf.keras.layers.Dropout(0.15),
            tf.keras.layers.Dense(3, activation='relu'),
            tf.keras.layers.Dropout(0.15),
            tf.keras.layers.Dense(1, activation='sigmoid')  # Sigmoid for binary classification
        ])
        print('Model Defined')

        custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        model_fcn_binary_lh.compile(
            optimizer=custom_optimizer,
            loss='binary_crossentropy',
            metrics=['accuracy']  
        )

        class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
        class_weight_dict = dict(enumerate(class_weights))

        print('Class Weight: ', class_weight_dict)

        model_fcn_binary_lh.fit(
            X_train_normalized, y_train,
            epochs=100,
            batch_size=32,
            class_weight=class_weight_dict,
            verbose=1  
        )

        thresholds = np.arange(0.01, 1.0, 0.01)
        y_test_pred_proba = model_fcn_binary_lh.predict(X_test_normalized)

        best_threshold = 0.0
        best_metric = 0.0

        for threshold in thresholds:
            y_test_pred_binary = (y_test_pred_proba > threshold).astype(int)
            metric_value = balanced_accuracy_score(y_test, y_test_pred_binary)

            if metric_value > best_metric:
                best_metric = metric_value
                best_threshold = threshold

        y_test_pred_proba = model_fcn_binary_lh.predict(X_test_normalized)

        if len(np.unique(y_test)) > 1:
            auc_score = roc_auc_score(y_test, y_test_pred_proba)
            auc_scores.append(auc_score)  
            if auc_score < 0.5:
                print(f"<0.5 AUC Score: {auc_score}")
                print("----------------------------------------")
            else:
                print(f"AUC Score: {auc_score}")

        else:
            auc_scores.append(None)  
            print(f"Skipping AUC computation for this fold as y_test contains only one class: {np.unique(y_test)}")

        y_test_pred_binary = (y_test_pred_proba > best_threshold).astype(int)
        balanced_acc = balanced_accuracy_score(y_test, y_test_pred_binary)
        balanced_accs.append(balanced_acc) 
        best_thresholds.append(best_threshold)  
        print(f"Balanced Accuracy: {balanced_acc}")

        pbar.update(1)

18


LOSO CV Progress:   0%|                                    | 0/18 [00:00<?]   0%WARNING:absl:At this time, the v2.11+ optimizer `tf.keras.optimizers.Adam` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.Adam`.


X and y train shapes: 
(282, 16)
(282,)
Train set - Class 0 (Low Stress): 131, Class 1 (High Stress): 151
Test set - Class 0 (Low Stress): 9, Class 1 (High Stress): 11
Test set size: 20
Train set shape: (282, 16)
Test set shape: (20, 16)
(282, 16)
(20, 16)
Model Defined
Class Weight:  {0: 1.0763358778625953, 1: 0.9337748344370861}
Epoch 1/100


2024-10-07 10:50:34.420921: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


9/9 [==============================] - 0s 696us/step - loss: 0.7035 - accuracy: 0.4326
Epoch 2/100
9/9 [==============================] - 0s 561us/step - loss: 0.6902 - accuracy: 0.5426
Epoch 3/100
9/9 [==============================] - 0s 552us/step - loss: 0.6949 - accuracy: 0.5142
Epoch 4/100
9/9 [==============================] - 0s 486us/step - loss: 0.6937 - accuracy: 0.5213
Epoch 5/100
9/9 [==============================] - 0s 487us/step - loss: 0.6920 - accuracy: 0.4681
Epoch 6/100
9/9 [==============================] - 0s 490us/step - loss: 0.6838 - accuracy: 0.5780
Epoch 7/100
9/9 [==============================] - 0s 486us/step - loss: 0.6837 - accuracy: 0.5674
Epoch 8/100
9/9 [==============================] - 0s 486us/step - loss: 0.6794 - accuracy: 0.5674
Epoch 9/100
9/9 [==============================] - 0s 481us/step - loss: 0.6792 - accuracy: 0.5638
Epoch 10/100
9/9 [==============================] - 0s 488us/step - loss: 0.6764 - accuracy: 0.5816
Epoch 11/100
9/9 [===

...

X and y train shapes: 
(251, 16)
(251,)
Train set - Class 0 (Low Stress): 100, Class 1 (High Stress): 151
Test set - Class 0 (Low Stress): 40, Class 1 (High Stress): 11
Test set size: 51
Train set shape: (251, 16)
Test set shape: (51, 16)
(251, 16)
(51, 16)
Model Defined


Class Weight:  {0: 1.255, 1: 0.8311258278145696}
Epoch 1/100
8/8 [==============================] - 0s 628us/step - loss: 0.7541 - accuracy: 0.4502
Epoch 2/100
8/8 [==============================] - 0s 591us/step - loss: 0.7083 - accuracy: 0.5020
Epoch 3/100
8/8 [==============================] - 0s 572us/step - loss: 0.6983 - accuracy: 0.4781
Epoch 4/100
8/8 [==============================] - 0s 523us/step - loss: 0.6942 - accuracy: 0.5060
Epoch 5/100
8/8 [==============================] - 0s 499us/step - loss: 0.6891 - accuracy: 0.5737
Epoch 6/100
8/8 [==============================] - 0s 492us/step - loss: 0.6924 - accuracy: 0.5339
Epoch 7/100
8/8 [==============================] - 0s 526us/step - loss: 0.6867 - accuracy: 0.5737
Epoch 8/100
8/8 [==============================] - 0s 519us/step - loss: 0.6977 - accuracy: 0.5857
Epoch 9/100
8/8 [==============================] - 0s 503us/step - loss: 0.6783 - accuracy: 0.6016
Epoch 10/100
8/8 [==============================] - 0s 513us

LOSO CV Progress: 100%|██████████████████████████████ | 18/18 [00:19<00:00] 100%


----------------------------------------
Balanced Accuracy: 0.5454545454545454


LOSO CV Progress: 100%|██████████████████████████████ | 18/18 [00:19<00:00] 100%


In [13]:
print('MLP: ')
print('Median, 25th Percentile, 75th Percentile: ')

if auc_scores:
    auc_scores_valid = [score for score in auc_scores if score is not None] 

    auc_median = np.median(auc_scores_valid)
    auc_25_percentile = np.percentile(auc_scores_valid, 25)
    auc_75_percentile = np.percentile(auc_scores_valid, 75)
    print(f"AUC Score - Median: {auc_median}, 25th Percentile: {auc_25_percentile}, 75th Percentile: {auc_75_percentile}")

balanced_acc_median = np.median(balanced_accs)
balanced_acc_25_percentile = np.percentile(balanced_accs, 25)
balanced_acc_75_percentile = np.percentile(balanced_accs, 75)

print(f"Balanced Accuracy - Median: {balanced_acc_median}, 25th Percentile: {balanced_acc_25_percentile}, 75th Percentile: {balanced_acc_75_percentile}")

MLP: 
Median, 25th Percentile, 75th Percentile: 
AUC Score - Median: 0.4022727272727272, 25th Percentile: 0.27083333333333337, 75th Percentile: 0.5555555555555556
Balanced Accuracy - Median: 0.5972222222222222, 25th Percentile: 0.5426136363636364, 75th Percentile: 0.6814732142857143
